# Exercise 2

Welcome to the second challenge! The goal of this exercise is to create a Event Manager Agent that is able to look at events in a Google calendar and create new ones. Part of the code is already provided for you, but you will find TODO comments that indicate where you should implement something. Good luck with the exercise!

## Setup
Let's start by importing the libraries that we will need.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os
from datetime import datetime
from typing import Optional

from crewai import Agent, Crew, Task
from crewai_tools import BaseTool
from dotenv import load_dotenv
from gcsa.event import Event
from gcsa.google_calendar import GoogleCalendar

Before you execute this cell, make sure to provide the environment variables `OPENAI_API_BASE`, `OPENAI_MODEL_NAME`, and `OPENAI_API_KEY` in the `.env` file.

In [ ]:
load_dotenv(override=True)

assert "OPENAI_MODEL_NAME" in os.environ, "No model specified in .env file!"
print("Using the following LLM model:", os.environ.get("OPENAI_MODEL_NAME"))

Before you execute the code below, make sure that you have the credentials for your Google Calendar ready. 

In [ ]:
gc = GoogleCalendar(credentials_path="../credentials/credentials.json")

print("Available calendars:")
for calendar in gc.get_calendar_list():
    print(f"Calendar: {calendar}, id: {calendar.calendar_id}")

In [ ]:
# TODO: replace [YOUR CALENDAR ID] with the id of the calendar that you want to interact with - for this exercise you can use "agents.wt24@gmail.com"
gc = GoogleCalendar("agents.wt24@gmail.com", credentials_path="../credentials/credentials.json")

## Tools

In [ ]:
# Used for the GetCurrentDateAndTimeTool
weekday_map = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

class GetEventsTool(BaseTool):
    name: str = "Get events within a certain time span from the calendar"
    description: str = (
        "Returns a list of dictionaries that store information about the events in the specified time span."
    )

    def _run(self, start_datetime_isoformat: str, end_datetime_isoformat: str) -> dict:
        start = datetime.fromisoformat(start_datetime_isoformat)
        end = datetime.fromisoformat(end_datetime_isoformat)

        events = gc.get_events(time_min=start, time_max=end)
        output = []

        for event in events:
            recurrence = event.recurrence[0] if event.recurrence else None
            output.append(
                {
                    "title": event.summary,
                    "description": event.description,
                    "location": event.location,
                    "start": event.start.isoformat(),
                    "end": event.end.isoformat(),
                    "recurrence": recurrence,
                }
            )

        return output

    def cache_function(*args):
        return False


class CreateEventTool(BaseTool):
    name: str = "Creates a new event in the calendar."
    description: str = "This tool can be used to create new events in the calendar."

    def _run(
        self,
        start_datetime_isoformat: str,
        end_datetime_isoformat: str,
        event_title: str,
        event_description: str,
        event_location: Optional[str] = None,
    ) -> None:
        start = datetime.fromisoformat(start_datetime_isoformat)
        end = datetime.fromisoformat(end_datetime_isoformat)

        event = Event(
            summary=event_title, description=event_description, start=start, end=end, location=event_location
        )

        gc.add_event(event)

    def cache_function(*args):
        return False


class GetCurrentDateAndTimeTool(BaseTool):
    name: str = "Get the current date and time"
    description: str = "Returns a dictionary with information about the current date and time."

    def _run(self) -> dict:
        now = datetime.now()
        return {
            "year": now.year,
            "month": now.month,
            "day": now.day,
            "hour": now.hour,
            "minute": now.minute,
            "weekday": weekday_map[now.weekday()],
        }

    def cache_function(*args):
        return False

## Agents

In [ ]:
event_manager = Agent(
    role="Senior Event Manager",
    goal=(
        "Use your event management expertise to help the user "
        "with his request. Make sure that you incorporate your "
        "domain knowldege. For example, you are aware that even "
        "if an event in a calendar is an all-day event, it does "
        "not always mean that the user is really blocked by this "
        "event the entire day. Make sure to infer as much as possible "
        "from the event information that you have access to. Here is "
        "the user's request: {user_request}"
    ),
    backstory="A knowledgeable expert in planning and managing events.",
    tools=[GetCurrentDateAndTimeTool(), GetEventsTool(), CreateEventTool()],
    cache=False,
    verbose=True,
)

## Tasks

In [ ]:
calendar_task = Task(
    description="Handle the user's request regarding his calendar. The language of your answer should match the language of the question.",
    expected_output="A concise and helpful answer to the user's request.",
    agent=event_manager,
)

## Kicking off the Crew

Now it's time to kick off the crew. This part is already implemented for you, so go ahead an execute it. You will be asked for a question, here are some inspirations:

- Ich hatte am Mittwoch vom 10 Uhr bis 12 Uhr einen Termin mit Herrn Müller, in dem es um die neue Strategie ging. Kannst du den Termin nachträglich in meinen Kalender eintragen?
- Habe ich morgen irgendwann zwischen 14 Uhr und 16 Uhr noch einen freien Slot für einen 30-minütigen Termin? Falls ja, bitte erstelle mir einen Blocker.

In [ ]:
crew = Crew(agents=[event_manager], tasks=[calendar_task], cache=False)

request = input("Question: ")

result = crew.kickoff(inputs={
    "user_request": request
})

print("reply:", result.raw)